In [1]:
# import os
# os.listdir('../input/rl-project')
# import sys
# sys.path.insert(0,'../input/rl-project/')

In [2]:
import torch
from torch.distributions import Categorical
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import time
import math
import cv2
from tqdm import tqdm

from IPython.display import clear_output

from torch.utils.tensorboard import SummaryWriter
import socket
from datetime import datetime
import os

from agents import Agent
from environment import SimulationEnvironment0
from replay_buffers import *
from utils import *

import copy
experiment_name='final_2_corr'

In [3]:
torch.autograd.set_detect_anomaly(False)

In [4]:
def compute_dist_discrete(actions):
    action_probs = actions.softmax(-1)
    dist = Categorical(action_probs)
    return dist

def compute_dist_continuous(actions):
    
    means = actions[:,:2]
    correlations = actions[:,2:-2].tanh()*(1-1e-5)
    variances = actions[:,-2:].exp()

    b,n = means.shape
    corr_matrix = torch.zeros(b, n, n, device=means.device)
    indices = torch.tril_indices(n, n, offset=-1)
    corr_matrix[..., indices[0], indices[1]] = correlations
    corr_matrix[..., indices[1], indices[0]] = correlations
    corr_matrix[..., torch.arange(n), torch.arange(n)] = 1
    cov_matrix = corr_matrix * variances[...,None] * variances[:,None]
    
    dist = torch.distributions.multivariate_normal.MultivariateNormal(means, cov_matrix)
    
    return dist

In [5]:
def decode_action_discrete(dist, sampled_action):
    # WARNING: uses global dec_x dec_y to decode the actions!
    log_prob = dist.log_prob(sampled_action)
    sampled_action_decoded = torch.stack([dec_x[sampled_action],dec_y[sampled_action]],1)
    return sampled_action_decoded, log_prob

def decode_action_continuous(dist, sampled_action):
    log_prob_sample = dist.log_prob(sampled_action)
    sampled_action_decoded = sampled_action.tanh()
    log_prob = log_prob_sample - torch.log( 1- sampled_action_decoded.square() + 1e-8 ).sum(-1) # correct accounting for the tanh transform
    return sampled_action_decoded, log_prob

In [30]:
def update_models(decay=1e-3):
    update_target_model(model=actor, target_model=target_actor,decay=decay)
    update_target_model(model=V, target_model=target_V, decay=decay)
    update_target_model(model=Q, target_model=target_Q, decay=decay)

def flatten_sequences(X, removelast=False):
    for i in range(len(X)):
        if removelast:
            X[i] = X[i][:,:-1]
        s = X[i].shape
        if len(s)==2:
            X[i] = X[i].reshape(s[0]*s[1])
        else:
            X[i] = X[i].reshape(s[0]*s[1],-1)
    return X

def reshape_sequences(X, shape):
    for i in range(len(X)):
        if X[i].shape.numel()==np.prod(shape):
            X[i] = X[i].reshape(shape)
        else:
            X[i] = X[i].reshape(shape+(-1,))
    return X

def get_batch(X, batch_size, b_idx):
    start = b_idx*batch_size
    end = (b_idx+1)*batch_size
    for i in range(len(X)):
        X[i] = X[i][start:end]
    return X

def initialize_zeros(shape, n, device):
    X=[]
    for _ in range(n):
        X.append(torch.zeros(shape, device=device))
    return X

def update_batched(X, U, batch_size, b_idx):
    start = b_idx*batch_size
    end = (b_idx+1)*batch_size
    for i in range(len(X)):
        X[i][start:end] = U[i]
    return X




1290

In [39]:
seed = 0

EXPERIMENTS = [#{"entropy": 3e-4, 'PPO':4, 'replay_ratio':0, 'training_steps':2**17},
               {"entropy": 3e-4, 'PPO':2, 'replay_ratio':1, 'training_steps':2**17},
               #{"entropy": 3e-4, 'PPO':2, 'replay_ratio':3, 'training_steps':2**17}
               ]


# simulation
num_simulations = 128
num_blackholes = 1

# agent
hidden_size = 512
simlog_res = 255
use_symlog = True
discrete_actions = False
simlog_half_res = simlog_res//2
simlog_max_range = 1
actions_res = 5
levels=2
input_type = 'complete'

lr = 3e-4
lr_actor = 3e-5



# training
#training_steps = 2**18
#epochs=8
lamb = 0.8
gamma = 0.98
smoothing = 1e-2
eps = 0.05 # for PPO update
seg_len = 2**5
h_samples = 0
c=0.1


# replay buffers
num_steps = 1024
batch_size = 2**10
replay_batch_size = num_simulations

plot = False

validate_every = 2**9

bin_values = (torch.arange(simlog_res)-simlog_half_res).cuda()/simlog_half_res*simlog_max_range
bin_values = bin_values.sign()*(bin_values.abs().exp()-1)

#dec_x, dec_y = torch.meshgrid(torch.arange(actions_res)/(actions_res-1)*2-1, torch.arange(actions_res)/(actions_res-1)*2-1)
#dec_x, dec_y = dec_x.flatten().cuda(), dec_y.flatten().cuda()

metric_idx = torch.pow(2,torch.arange(15))-1

fig, ax = plt.subplots(figsize=(10,10))



for experiment in EXPERIMENTS:

    # SETUP EXPERIMENT

    # set seed
    torch.manual_seed(seed)

    # initialize logging (with tensorboard)
    current_time = datetime.now().strftime("%b%d_%H-%M-%S")
    log_dir = os.path.join(
        "runs",experiment_name, current_time + "_" + socket.gethostname() 
    )
    tb_writer = SummaryWriter(log_dir)

    # initialize the simulation
    sim = SimulationEnvironment0(num_simulations=num_simulations,
                            num_blackholes=num_blackholes, 
                            force_constant=0.002, 
                            velocity_scale=0.01,
                            goal_threshold=0.05,
                            max_steps=250,
                            device='cuda')
    states = sim.get_state()

    # initialize the networks
    if discrete_actions:
        action_dim = actions_res**2
        compute_dist = compute_dist_discrete
        decode_action = decode_action_discrete
    else:
        action_dim = 5
        compute_dist = compute_dist_continuous
        decode_action = decode_action_continuous

    if use_symlog:
        actor = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, critic=False, action_dimension=action_dim).cuda()
        actor2 = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, critic=False, action_dimension=action_dim).cuda()
        V = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, actor=False, value_dimension=simlog_res).cuda()
        Q = Agent(hidden_size + 2, hidden_size, levels, input_type='base', actor=False, value_dimension=simlog_res).cuda()
    else:
        actor = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, critic=False, action_dimension=action_dim).cuda()
        actor2 = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, critic=False, action_dimension=action_dim).cuda()
        V = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, actor=False, value_dimension=1).cuda()
        Q = Agent(hidden_size + 2, hidden_size, levels, input_type='base', actor=False, value_dimension=1).cuda()

    optim_actor = torch.optim.AdamW(list(actor.parameters())+list(actor2.parameters()), lr=lr_actor, weight_decay=1e-3)
    optim_critic = torch.optim.AdamW(list(V.parameters())+list(Q.parameters()), lr=lr, weight_decay=1e-3)
    target_actor = copy.deepcopy(actor)
    target_V = copy.deepcopy(V)
    target_Q = copy.deepcopy(Q)

    # initialize the replay buffer
    replay_buffer = Replay_Buffer_Segments(state_shape=((num_blackholes+2),2), action_shape=(2,), params_shape=(action_dim,), segment_lenght=seg_len, num_simulations=num_simulations, num_steps=num_steps, batch_size=replay_batch_size, device='cuda')

    # initialize vailidation plane
    x,y = torch.meshgrid(torch.arange(100),torch.arange(100))
    pos = torch.stack([x.flatten(), y.flatten()],1)/100

    target_pos = torch.ones_like(pos)*0.25      # position of the target
    bh_pos = torch.ones_like(pos)*0.75          # position of the blackholes

    st=torch.stack([pos,target_pos,bh_pos],1)


    # START EXPERIMENT
    pbar = tqdm(range(experiment['training_steps']))
    for i in pbar:
        t0=time.time()

        # GENERATE EXPERIENCE
        with torch.inference_mode():
            
            states = states.reshape(states.shape[0],-1).cuda()

            # compute action distribution according to the current policy
            actions, _, _ = actor(states)
            dist = compute_dist(actions)

            # sample an action
            sampled_action = dist.sample()
            sampled_action_decoded, log_prob = decode_action(dist, sampled_action)

            # simulation step
            rewards, new_states, terminals = sim.step(sampled_action_decoded)

            # save experience
            replay_buffer.add_experience(states.reshape(new_states.shape), sampled_action, actions, log_prob.exp(), rewards, terminals)

        tb_writer.add_scalar('Reward',rewards.mean().item(), i)

        if (i+1) % seg_len == 0:

            # ON POLICY UPDATE

            seg_state, seg_actions, _, seg_action_probs, seg_rewards, seg_terminal = replay_buffer.get_last_segment()

            with torch.inference_mode():

                # get target values for all the states in the segment
                V_t = torch.zeros(num_simulations*seg_len, device='cuda')
                for b_idx in range((seg_len*num_simulations+batch_size-1)//batch_size):

                    b_state = seg_state.reshape((seg_len)*num_simulations,-1)[b_idx*batch_size:(b_idx+1)*batch_size]
                    
                    _, Vs, _= target_V(b_state)
                    V_t[b_idx*batch_size:(b_idx+1)*batch_size] = (torch.softmax(Vs,1)@bin_values[:,None])[:,0]
        
                V_t = V_t.reshape(num_simulations, seg_len)

                # compute GAE and TD lambda returns
                gae = torch.zeros_like(V_t)
                for t in reversed(range(seg_len-1)):

                    d_t = -V_t[:,t] + seg_rewards[:,t] + gamma*V_t[:,t+1]*seg_terminal[:,t].logical_not()

                    gae[:,t] = d_t + gamma*lamb*gae[:,t+1]*seg_terminal[:,t].logical_not()

                tdl = V_t + gae

            # unite batch and sequence dimensions
            [gae, tdl, seg_state, seg_actions, seg_action_probs] = flatten_sequences([gae, tdl, seg_state, seg_actions, seg_action_probs], removelast=True)

            # run PPO for n epochs
            for ppo_epoch in range(experiment['PPO']):
                for b_idx in range((seg_len*num_simulations+batch_size-1)//batch_size):

                    # get batch
                    [b_gae, b_tdl, b_state, b_action, b_prob] = get_batch([gae, tdl, seg_state, seg_actions, seg_action_probs], batch_size, b_idx)

                    # compute action distribution according to the updated policy
                    actions, _, _ = actor(b_state)
                    dist = compute_dist(actions)

                    # compute probability of the chosen action with respect to the updated policy
                    sampled_action_decoded, log_prob = decode_action(dist, b_action)

                    # compute the value functions
                    _, Vs, hs = V(b_state)
                    _, Qs, _ = Q(torch.concat([hs, sampled_action_decoded],-1))

                    # compute PPO clip objective
                    r = (log_prob.exp() + 1e-5)/(b_prob + 1e-5)
                    L = torch.minimum(b_gae*r, b_gae*r.clip(1-eps,1+eps))

                    # approximation of the hentropy
                    H = -log_prob.exp()

                    actor_error = - L - H*experiment['entropy']

                    y = two_hot_encode(b_tdl, simlog_max_range, simlog_res, simlog_half_res, smoothing=smoothing)
                    critic_error_V = torch.nn.functional.cross_entropy(Vs, y, reduction='none')
                    critic_error_Q = torch.nn.functional.cross_entropy(Vs+Qs, y, reduction='none')
                    critic_error = (critic_error_V + critic_error_Q) / 2
                    error = actor_error + critic_error

                    optim_critic.zero_grad()
                    optim_actor.zero_grad() 
                    error.mean().backward()
                    optim_critic.step()
                    optim_actor.step()
                
                update_models(decay=5e-3)

            tb_writer.add_scalar('critic_error_V',critic_error_V.mean().item(), i)
            tb_writer.add_scalar('critic_error_Q',critic_error_Q.mean().item()-critic_error_V.mean().item(), i)
            tb_writer.add_scalar('hentropy',H.mean().item(), i)

            # REPLAY EXPERIENCES

            # replay experiences
            for replay_epoch in range(experiment['replay_ratio']):

                seg_state, seg_actions, seg_action_params, seg_action_probs, seg_rewards, seg_terminal = replay_buffer.get_batch()

                with torch.inference_mode():

                    # get target values and action probs for all the states in the segment

                    V_t, Q_t, probs_t, Q_t_corr, probs_corr, probs_seg_corr = initialize_zeros(replay_batch_size*seg_len, n=6, device='cuda')
                    action_corr = torch.zeros(replay_batch_size*seg_len, 2, device='cuda')

                    for b_idx in range((seg_len*replay_batch_size+batch_size-1)//batch_size):

                        # get batch
                        [b_state, b_action, b_seg_action_params] = get_batch(flatten_sequences([seg_state, seg_actions, seg_action_params]), batch_size, b_idx)

                        # compute action distribution according to current policy
                        actions, _, _ = target_actor(b_state)
                        dist = compute_dist(actions)

                        # compute the probability of the replayed action
                        sampled_action_decoded, log_prob = decode_action(dist, b_action)

                        # sample a new action from the current policy (for the bias correction)
                        sampled_action_corr = dist.sample()
                        sampled_action_decoded_corr, log_prob_corr = decode_action(dist, sampled_action_corr)

                        # compute the probabilty of the action sampled with the current policy with respect to the old policy (for the ro in bias correction)
                        seg_dist = compute_dist(b_seg_action_params)
                        _, seg_log_prob_corr = decode_action(seg_dist, sampled_action_corr)

                        # compute the value functions
                        _, Vs, hs = target_V(b_state)
                        _, Qs, _ = target_Q(torch.concat([hs, sampled_action_decoded],-1))
                        _, Qs_corr, _ = target_Q(torch.concat([hs, sampled_action_decoded_corr],-1))

                        b_V_t = (torch.softmax(Vs,1)@bin_values[:,None])[:,0]
                        b_Q_t = (torch.softmax(Vs+Qs,1)@bin_values[:,None])[:,0]

                        b_Q_t_corr = (torch.softmax(Vs+Qs_corr,1)@bin_values[:,None])[:,0]
                        b_action_corr = sampled_action_corr

                        update_batched( [V_t, Q_t, probs_t, Q_t_corr, action_corr, probs_corr, probs_seg_corr],
                                        [b_V_t, b_Q_t, log_prob.exp(), b_Q_t_corr, b_action_corr, log_prob_corr.exp(), seg_log_prob_corr.exp()],
                                        batch_size, b_idx)

                    # reshape to expose the sequences
                    [V_t, Q_t, probs_t, Q_t_corr, action_corr, probs_corr, probs_seg_corr] = reshape_sequences([V_t, Q_t, probs_t, Q_t_corr, action_corr, probs_corr, probs_seg_corr], (replay_batch_size, seg_len))


                    # compute targets (as in RETRACE)
                    Q_ret = torch.zeros_like(V_t)
                    V_target = torch.zeros_like(V_t)

                    Q_ret[:,-1] = Q_t[:,-1]

                    for t in reversed(range(seg_len-1)):

                        ro = (probs_t[:,t+1] + 1e-5)/(seg_action_probs[:,t+1] + 1e-5) # ro of t+1
                        ci = lamb * torch.minimum(torch.ones(1, device='cuda'), ro)

                        Q_ret[:,t] = seg_rewards[:,t] + gamma*(ci*(Q_ret[:,t+1] - Q_t[:,t+1]) + V_t[:,t+1])
                        V_target[:, t+1] = ci*(Q_ret[:,t+1] - Q_t[:,t+1]) + V_t[:,t+1]

                    V_target[:, 0] = ci*(Q_ret[:,0] - Q_t[:,0]) + Q_t_corr[:,0]

                # unite batch and sequence dimensions
                [Q_ret, V_target, seg_state, seg_actions, probs_t, seg_action_probs] = flatten_sequences([Q_ret, V_target, seg_state, seg_actions, probs_t, seg_action_probs], removelast=True)
                [Q_t_corr, V_t, action_corr, probs_corr, probs_seg_corr] = flatten_sequences([Q_t_corr, V_t, action_corr, probs_corr, probs_seg_corr], removelast=True)

                # run PPO for n epochs
                for ppo_epoch in range(experiment['PPO']):
                    for b_idx in range((seg_len*replay_batch_size+batch_size-1)//batch_size):

                        # get batch
                        [b_Q_ret, b_V_target, b_state, b_action, b_prob, b_prob_seg] = get_batch([Q_ret, V_target, seg_state, seg_actions, probs_t, seg_action_probs], batch_size, b_idx)
                        [b_Q_t_corr, b_V_t, b_action_corr, b_probs_corr, b_probs_seg_corr] = get_batch([Q_t_corr, V_t, action_corr, probs_corr, probs_seg_corr], batch_size, b_idx) # bias correction

                        # compute action distribution according to current policy
                        actions, _, _ = actor(b_state)
                        dist = compute_dist(actions)

                        # compute probability of the chosen action with respect to the updated policy
                        sampled_action_decoded, log_prob = decode_action(dist, b_action)

                        # compute probability of the bias-correction action with respect to the updated policy
                        _, log_prob_corr = decode_action(dist, b_action_corr)

                        # compute the value functions
                        _, Vs, hs = V(b_state)
                        _, Qs, _ = Q(torch.concat([hs, sampled_action_decoded],-1))

                        # compute the PPO objective (with respect to the target policy) and apply off policy correction (with bias correction)
                        r = (log_prob.exp() + 1e-5 )/(b_prob + 1e-5 )
                        adv = (b_prob/b_prob_seg).clip(max = c)*(b_Q_ret - b_V_target)

                        r_c = (log_prob_corr.exp() + 1e-5 )/(b_probs_corr + 1e-5)
                        adv_c = (1-c*b_probs_seg_corr/b_probs_corr).relu()*(b_Q_t_corr - b_V_t)

                        L = torch.minimum(adv*r, adv*r.clip(1-eps,1+eps)) #+ torch.minimum(adv_c*r_c, adv_c*r_c.clip(1-eps,1+eps))

                        # approximation of the hentropy
                        H = -log_prob.exp()

                        actor_error = - L - H*experiment['entropy']

                        y_V = two_hot_encode(b_V_target, simlog_max_range, simlog_res, simlog_half_res, smoothing=smoothing)
                        y_Q = two_hot_encode(b_Q_ret, simlog_max_range, simlog_res, simlog_half_res, smoothing=smoothing)
                        critic_error_V = torch.nn.functional.cross_entropy(Vs, y_V, reduction='none')
                        critic_error_Q = torch.nn.functional.cross_entropy(Vs+Qs, y_Q, reduction='none')
                        critic_error = (critic_error_V + critic_error_Q) / 2
                        error = actor_error + critic_error

                        optim_critic.zero_grad()
                        optim_actor.zero_grad() 
                        error.mean().backward()
                        optim_critic.step()
                        optim_actor.step()

                    update_models(decay=5e-3)


        states = new_states

        if (i+1) % validate_every == 0:
            # compute the value function and the action probability in each point of the validation plane
            Values = []
            V_t = []
            A = []
            for b in range((len(st)+batch_size-1)//batch_size):
                stb = st[b*batch_size:(b+1)*batch_size]
                _, v, _ = V(stb.reshape(stb.shape[0],-1).cuda())
                _, vt, _ = target_V(stb.reshape(stb.shape[0],-1).cuda())
                a, _, _ = actor(stb.reshape(stb.shape[0],-1).cuda())
                Values.append(v)
                V_t.append(vt)
                A.append(a)
            Values = torch.concat(Values,0)
            A = torch.concat(A,0)
            V_t = torch.concat(V_t,0)

            # decode values
            if use_symlog:
                Values = (Values.softmax(1)@bin_values[:,None])[:,0].detach().cpu()
                V_t = (V_t.softmax(1)@bin_values[:,None])[:,0].detach().cpu()
            else:
                V = Values.cpu()
                V_t = V_t.cpu()

            tb_writer.add_image('V', (Values.reshape(1,100,100)/2+0.5), i)
            tb_writer.add_image('V_t', V_t.reshape(1,100,100)/2+0.5, i)

            # plot the mean.tanh() (not the real mean of the distribution of the tanh)
            pos = st[:,0].reshape(100,100,-1)[::2,::2]
            A = A.reshape(100,100,-1)[::2,::2]

            plt.quiver(pos[...,1].flatten(), -pos[...,0].flatten(), A[...,1].tanh().detach().cpu().flatten(), -A[...,0].tanh().detach().cpu().flatten(), color='g',scale=50, headwidth=2)
            ax.axis('off')
            plt.gca().set_aspect('equal')
            plt.subplots_adjust(0,0,1,1,0,0)
            fig.canvas.draw()
            data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            plt.clf()

            tb_writer.add_image('Policy visualization', np.transpose(data,(2,0,1)) , i)

 55%|█████▍    | 71583/131072 [19:44<16:24, 60.41it/s] 


KeyboardInterrupt: 

<Figure size 1000x1000 with 0 Axes>